# Using EcoFOCIpy to process raw field data

## DY2406 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2024/CTDcasts/dy2406/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2406' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2406.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2406/rawconverted/CTD001.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['CTD001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,prdm,par,scan,datetime
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11
mean,7.589064,102.600091,330.497091,32.128664,32.126982,25.508173,25.506864,1.198509,3.942727,3.942382,49.937818,0.007262,8802.818182,2024-05-09 09:13:10.454545408
min,7.567100,102.300000,329.540000,32.123000,32.121300,25.504000,25.502700,1.134900,3.939400,3.939000,48.533000,0.000348,8590.000000,2024-05-09 09:13:02
25%,7.580650,102.487500,330.129500,32.127250,32.125600,25.507100,25.505850,1.146050,3.941950,3.941600,49.178000,0.001243,8705.000000,2024-05-09 09:13:06
50%,7.585800,102.543000,330.354000,32.129700,32.128000,25.508900,25.507600,1.193400,3.943500,3.943200,49.923000,0.005654,8806.000000,2024-05-09 09:13:11
75%,7.601000,102.765500,331.016500,32.130600,32.129300,25.509600,25.508600,1.223800,3.943850,3.943550,50.833000,0.013758,8898.000000,2024-05-09 09:13:14.500000
max,7.612900,102.926000,331.536000,32.131300,32.129700,25.510100,25.508800,1.355600,3.944400,3.944100,51.224000,0.017131,9019.000000,2024-05-09 09:13:19
std,0.014692,0.201798,0.640407,0.002700,0.002847,0.002000,0.002097,0.065823,0.001613,0.001686,0.989103,0.006712,136.012366,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2406',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2024, 5, 7),
 'EndDate': datetime.date(2024, 5, 10),
 'Project': 'Spring Mooring Cruise',
 'ChiefScientist': 'Shaun Bell',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Mooring Survey',
 'CruiseYear': 2024,
 'ctdlogs_pdf_name': 'DY2406_CTDCastLogs.pdf'}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54719,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2406',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2406',
 'Project': 'Spring Mooring Cruise',
 'StationNo_altname': '1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 54,
 'LatitudeMin': 16.54,
 'LongitudeDeg': 164,
 'LongitudeMin': 51.8,
 'GMTDay': 9,
 'GMTMonth': 'May',
 'GMTYear': 2024,
 'GMTTime': 62160,
 'DryBulb': 4.1,
 'RelativeHumidity': 58,
 'WetBulb': -99.9,
 'Pressure': 1012,
 'SeaState': '',
 'Visibility': '',
 'WindDir': 314,
 'WindSpd': 3.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 59,
 'StationNameID': 'UPP-3A',
 'MaxDepth': 50,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'WaterMassCode': 'G',
 'isMooringProfile

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [10]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['CTD001.btl'] = cruise_data['CTD001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['CTD001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,Pressure [dbar],par,scan,datetime,cast
bottle,,,,,,,,,,,,,,,
4.0,7.5705,102.348,329.687,32.129,32.1265,25.5085,25.5066,1.1934,3.9426,3.9418,48.858,0.015487,8726.0,2024-05-09 09:13:07,ctd001


In [11]:
cruise_data['CTD001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'Pressure [dbar]', 'par', 'scan',
       'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [12]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [14]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['CTD001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [15]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 11)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
Data variables: (12/15)
    oxy_concM_ch1       (bottle) float64 7.606 7.583 7.59 ... 7.584 7.586 7.613
    oxy_percentsat_ch1  (bottle) float64 102.8 102.5 102.6 ... 102.5 102.5 102.9
    oxy_conc_ch1        (bottle) float64 331.2 330.2 330.5 ... 330.3 330.4 331.5
    salinity_ch1        (bottle) float64 32.13 32.13 32.13 ... 32.13 32.12 32.13
    salinity_ch2        (bottle) float64 32.13 32.13 32.13 ... 32.12 32.12 32.13
    sigma_t_ch1         (bottle) float64 25.51 25.51 25.51 ... 25.51 25.5 25.51
    ...                  ...
    temperature_ch2     (bottle) float64 3.944 3.944 3.943 ... 3.94 3.939 3.943
    Pressure [dbar]     (bottle) float64 50.67 50.4 49.5 ... 48.53 48.69 51.03
    par                 (bottle) float64 0.002342 0.0004283 ... 0.0004911
    scan                (bottle) float64 8.59e+03 8.641e+03 ... 9.019e+03
    datetime            (bottle) datetime64[ns] 2024-05-09T09:13:02 ... 2024-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [16]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [17]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['CTD001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [18]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 11, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/15)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 7.606 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 102.8 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 331.2 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.13 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.13 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.51 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 3.944 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 50.67 ......
    par                 (latitude, longitude, time, bottle) float64 0.002342 ...
    scan                (latitude, longitude, time, bottle) float64 8.59e+03 ...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [19]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 11, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0 11.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/15)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 7.606 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 102.8 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 331.2 ......
    salinity_ch1        (latitude, longitude, time, bottle) float64 32.13 ......
    salinity_ch2        (latitude, longitude, time, bottle) float64 32.13 ......
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.51 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 3.944 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 50.67 ......
    par                 (latitude, longitude, time, bottle) float64 0.002342 ...
    scan                (latitude, longitude, time, bottle) float64 8.59e+03 ...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      DY2406
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-12-27T11:00:26Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [20]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history